<a href="https://colab.research.google.com/gist/PrasangaDhungel/4c8827c1159c0258c2ae442aa0d28c9f/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()
# from google.colab import drive
import tensorboardcolab 
import numpy as np
import os
import time

In [0]:
# drive.mount('/content/gdrive')
# tbc= tensorboardcolab.TensorBoardColab()

In [7]:
text = open("poems.txt", 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 11556 characters


In [8]:
print(text[:500])

संसार रूपी सुख स्वर्गभित्र,
रमें रमाएँ लिइ भित्र चित्र ।
सारा भयो त्यो मरुभूभि तुल्य,
रातै परेझैं अब बुझ्छु बल्ल ।
रहेछ संसार निशा समान,
आएन ज्यूँदै रहँदा नि ज्ञान ।
आखीर श्रीकृष्ण रहेछ एक,
न भक्ति भो, ज्ञान, नभो विवेक ।
महामरुमा कण झैं म तातो,
जलेर मर्दो बिनु आश लाटो ।
सुकी रहेको तरु झैं छु खाली,
चिताग्नि तापी जल डाल्न फाली ।
संस्कार आफ्नो सब नै गुमाएँ,
म शून्यमा शून्य सरी बिलाएँ ।
जन्में म यो स्वर्गविषे पलाएँ,
आखीर भै खाक त्यसै बिलाएँ

कुन मन्दिरमा जान्छौ यात्री,
कुन मन्दिरमा जाने हो ?
कुन साम


In [11]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

71 unique characters


**Vectorize the Text** <br>
Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [12]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
print(char2idx, idx2char)

text_as_int = np.array([char2idx[c] for c in text])

{'\n': 0, ' ': 1, '!': 2, '(': 3, ')': 4, ',': 5, '-': 6, ':': 7, '?': 8, 'ँ': 9, 'ं': 10, 'अ': 11, 'आ': 12, 'इ': 13, 'ई': 14, 'उ': 15, 'ऊ': 16, 'ए': 17, 'ऐ': 18, 'ओ': 19, 'औ': 20, 'क': 21, 'ख': 22, 'ग': 23, 'घ': 24, 'ङ': 25, 'च': 26, 'छ': 27, 'ज': 28, 'झ': 29, 'ञ': 30, 'ट': 31, 'ठ': 32, 'ड': 33, 'ढ': 34, 'ण': 35, 'त': 36, 'थ': 37, 'द': 38, 'ध': 39, 'न': 40, 'प': 41, 'फ': 42, 'ब': 43, 'भ': 44, 'म': 45, 'य': 46, 'र': 47, 'ल': 48, 'व': 49, 'श': 50, 'ष': 51, 'स': 52, 'ह': 53, 'ा': 54, 'ि': 55, 'ी': 56, 'ु': 57, 'ू': 58, 'ृ': 59, 'े': 60, 'ै': 61, 'ो': 62, 'ौ': 63, '्': 64, '।': 65, '–': 66, '‘': 67, '’': 68, '“': 69, '”': 70} ['\n' ' ' '!' '(' ')' ',' '-' ':' '?' 'ँ' 'ं' 'अ' 'आ' 'इ' 'ई' 'उ' 'ऊ' 'ए'
 'ऐ' 'ओ' 'औ' 'क' 'ख' 'ग' 'घ' 'ङ' 'च' 'छ' 'ज' 'झ' 'ञ' 'ट' 'ठ' 'ड' 'ढ' 'ण'
 'त' 'थ' 'द' 'ध' 'न' 'प' 'फ' 'ब' 'भ' 'म' 'य' 'र' 'ल' 'व' 'श' 'ष' 'स' 'ह'
 'ा' 'ि' 'ी' 'ु' 'ू' 'ृ' 'े' 'ै' 'ो' 'ौ' '्' '।' '–' '‘' '’' '“' '”']


In [13]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '(' :   3,
  ')' :   4,
  ',' :   5,
  '-' :   6,
  ':' :   7,
  '?' :   8,
  'ँ' :   9,
  'ं' :  10,
  'अ' :  11,
  'आ' :  12,
  'इ' :  13,
  'ई' :  14,
  'उ' :  15,
  'ऊ' :  16,
  'ए' :  17,
  'ऐ' :  18,
  'ओ' :  19,
  ...
}


In [14]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'संसार रूपी सु' ---- characters mapped to int ---- > [52 10 52 54 47  1 47 58 41 56  1 52 57]


**Prediction**<br>
Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step. <br>

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


**Create training examples and targets**

Dividing the text into example sequences so that each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 9 and our text is "प्रकृतिले". The input sequence would be "प्रकृति", and the target sequence "रकृतिले".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [15]:
str = 'प्रकृतिले'
len(str)

9

In [16]:
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

स
ं
स
ा
र


In [17]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'संसार रूपी सुख स्वर्गभित्र,\nरमें रमाएँ लिइ भित्र चित्र ।\nसारा भयो त्यो मरुभूभि तुल्य,\nरातै परेझैं अब '
'बुझ्छु बल्ल ।\nरहेछ संसार निशा समान,\nआएन ज्यूँदै रहँदा नि ज्ञान ।\nआखीर श्रीकृष्ण रहेछ एक,\nन भक्ति भो, '
'ज्ञान, नभो विवेक ।\nमहामरुमा कण झैं म तातो,\nजलेर मर्दो बिनु आश लाटो ।\nसुकी रहेको तरु झैं छु खाली,\nचिता'
'ग्नि तापी जल डाल्न फाली ।\nसंस्कार आफ्नो सब नै गुमाएँ,\nम शून्यमा शून्य सरी बिलाएँ ।\nजन्में म यो स्वर्ग'
'विषे पलाएँ,\nआखीर भै खाक त्यसै बिलाएँ\n\nकुन मन्दिरमा जान्छौ यात्री,\nकुन मन्दिरमा जाने हो ?\nकुन सामग्री '


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [19]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'संसार रूपी सुख स्वर्गभित्र,\nरमें रमाएँ लिइ भित्र चित्र ।\nसारा भयो त्यो मरुभूभि तुल्य,\nरातै परेझैं अब'
Target data: 'ंसार रूपी सुख स्वर्गभित्र,\nरमें रमाएँ लिइ भित्र चित्र ।\nसारा भयो त्यो मरुभूभि तुल्य,\nरातै परेझैं अब '


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "प्" and tries to predict the index for "प्र" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.


In [20]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 52 ('स')
  expected output: 10 ('ं')
Step    1
  input: 10 ('ं')
  expected output: 52 ('स')
Step    2
  input: 52 ('स')
  expected output: 54 ('ा')
Step    3
  input: 54 ('ा')
  expected output: 47 ('र')
Step    4
  input: 47 ('र')
  expected output: 1 (' ')


### Create training batches

`tf.data` to split the text into manageable sequences and shuffle the data and pack it into batches

In [21]:
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  """
  To build a tf.Keras model
  Arguments:
  vocab_size : length of vocabulary in chars
  embedding_dim : The embedding dimension
  rnn_units : Number of RNN units
  batch_size : Batch size
  Returns:
  model : tf.Keras model
  """
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [41]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 71) # (batch_size, sequence_length, vocab_size)


In [42]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           18176     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 71)            72775     
Total params: 4,029,255
Trainable params: 4,029,255
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [44]:
sampled_indices

array([21, 63, 28, 56, 29, 23, 33, 45, 19, 64, 62, 47, 24,  6, 64,  8, 45,
       24,  8, 41, 24, 39, 12, 54, 32, 58, 18,  5, 22, 59, 27, 23, 13, 53,
       69, 60, 55, 48, 42, 51,  2, 43,  6, 31, 52, 49,  6,  3, 34, 36,  8,
       65, 29, 63, 53, 29, 62, 61, 45, 43,  3,  7, 46, 51, 17, 11, 31, 48,
       12, 26,  6,  5, 14, 11, 17, 47, 28, 49, 50, 15, 23, 14,  0, 66,  9,
       51, 46, 41, 32, 11, 55, 61, 48, 50, 65,  6,  2, 29, 31,  0])

In [45]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'यस खोलाका लहराभित्र\nलाग्छन् चिचिल्ला मन–रसका !\nप्रकृतिरानीको राज्य पवित्र !\nशहर–उपल्ला यी स्तरमा छन्'

Next Char Predictions: 
 'कौजीझगडमओ्ोरघ-्?मघ?पघधआाठूऐ,खृछगइह“ेिलफष!ब-टसव-(ढत?।झौहझोैमब(:यषएअटलआच-,ईअएरजवशउगई\n–ँषयपठअिैलश।-!झट\n'


In [46]:
def loss(labels, logits):
  """
  Return the loss of a model
  Arguments:
  labels: Actual Output vector of vocab_size * 1
  logits: Generated output vector of vocab_size * 1
  """
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 71)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.261596


In [0]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [48]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

tb_callback = tf.keras.callbacks.TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=1,
                         write_images=True)

W0614 12:42:57.502540 140028696885120 callbacks_v1.py:135] Weight and gradient histograms not supported for eagerexecution, setting `histogram_freq` to `0`.


In [49]:
EPOCHS=3000
print(EPOCHS, steps_per_epoch)

3000 1


In [0]:
history = model.fit(dataset.repeat(), epochs=EPOCHS,verbose=1, steps_per_epoch=steps_per_epoch, callbacks=[tb_callback ,checkpoint_callback])

Epoch 1/3000
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
1/1 [==============================] - 1s 1s/step - loss: 4.2639
Epoch 2/3000
1/1 [==============================] - 0s 290ms/step - loss: 4.2352
Epoch 3/3000
1/1 [==============================] - 0s 283ms/step - loss: 4.1846
Epoch 4/3000
1/1 [==============================] - 0s 294ms/step - loss: 3.9661
Epoch 5/3000
1/1 [==============================] - 0s 312ms/step - loss: 11.5496
Epoch 6/3000
1/1 [==============================] - 0s 313ms/step - loss: 5.8007
Epoch 7/3000
1/1 [==============================] - 0s 303ms/step - loss: 3.9888
Epoch 8/3000
1/1 [==============================] - 0s 301ms/step - loss: 4.1203
Epoch 9/3000
1/1 [==============================] - 0s 308ms/step - loss: 4.1532
Epoch 10/3000
1/1 [==============================] - 0s 322ms/step - loss: 4.1602
Epoch 11/3000
1/1 [==============================] - 0s 456m

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt'

In [50]:
print(vocab_size, embedding_dim, rnn_units)

71 256 1024


In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
# model.load_weights("laxmiprasad.h5")
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [52]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            18176     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 71)             72775     
Total params: 4,029,255
Trainable params: 4,029,255
Non-trainable params: 0
_________________________________________________________________


In [55]:
str = input("Want to save model?[y/n]")
if(str == "y" or str == "Y"):
  model.save("laxmiprasad.h5")

Want to save model?[y/n]y
